In [5]:
!pip install opencv-python # Required for cv2 module to be found.

     |████████████████████████████████| 26.6MB 19kB/s  eta 0:00:01     |██████████████████████████      | 21.5MB 533kB/s eta 0:00:10
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import pickle
import tarfile
import numpy as np
import scipy as sc
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import matplotlib.pyplot as plt

In [6]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

--2019-08-16 00:55:29--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving webproxy (webproxy)... 10.36.19.1
Connecting to webproxy (webproxy)|10.36.19.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  1.43MB/s    in 1m 40s  

2019-08-16 00:57:10 (1.62 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [10]:
# Extract dataset.
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close

<bound method TarFile.close of <tarfile.TarFile object at 0x7f536f29b588>>

In [10]:
# Read in first data_batch.
with open("cifar-10-batches-py/data_batch_1", "rb") as fo:
    data_batch = pickle.load(fo, encoding="bytes")

In [11]:
# Obtain the numpy array for the values of all the pixels in the data set and also the array of labels telling us what numpy array set is what type of picture.
image_height = 32
image_width = 32
pixels = data_batch[b"data"].reshape(len(data_batch[b"labels"]), 3, image_width, image_height)
labels = data_batch[b"labels"]

In [13]:
# Median filter operation to get rid of noise in the picture.
rgb = 3
for i in range(len(pixels)):
    for j in range(rgb):
        final = sc.ndimage.filters.median_filter(pixels[i][j], size = (3, 3))
        pixels[i][j] = final

In [14]:
# Adaptive histogram equalisation to sharpen contrast of the image while trying not to lose information with the blurring of lines/features
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
for i in range(len(pixels)):
    for j in range(rgb):
        final = clahe.apply(pixels[i][j])
        pixels[i][j] = final

In [23]:
# Restting the tensorflow graph just in case it still is holding left over info, also reading in the test set from the dataset so we can use our trained model to predict training set pictures.
tf.reset_default_graph()

with open("cifar-10-batches-py/test_batch", "rb") as fo:
    test_set = pickle.load(fo, encoding="bytes")

test_pixels = data_batch[b"data"].reshape(len(data_batch[b"labels"]), 3, image_width, image_height)
test_labels = data_batch[b"labels"]

x_train = pixels
y_train = labels
x_test = test_pixels
y_test = test_labels

In [24]:
# Intensity/brightness of each pixel will range from 1-255 and this is a problem for out selu function, as the values of the pixel increase then the function output linear increases. 
# This activation weight is large will result in our model beind dependant on his weigth when looking at unseen data, this is overfitting.
x_train = pixels.astype("float32")
x_test = x_test.astype("float32")

mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train - mean)/(std + 1e-7)
x_test = (x_test - mean)/(std + 1e-7)

In [26]:
# Chaning our train/test labels to an adjacency matrix.
num_classes = 10

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [27]:
# Tensorflow expects to read in the training/test set in this orientation so transpose the data to make them valid inputs
x_train = x_train.transpose(0, 2, 3, 1)
x_test = x_test.transpose(0, 2, 3, 1)

In [28]:
# Calls the sequential model API which allows use to create a model and add tensor and fully connected layers one by one. Each layer creates a weght when learning and the next layer builds on the previous layers weights to better generalise the training data.
model = tf.keras.models.Sequential()

In [29]:
# This is adding the first convolutional layer to our sequential model. 
# Convolutional layers are similar to the moving window transform used in median filtering, we take a kernal_size in this case [3, 3] and dot product with every value in the input resulting in weights, 
# we then use these weights to try find some feature unique to the input that we can learn from and generalise to other inputs that have similar features. 32 = filter size.
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", input_shape = x_train.shape[1:]))
# Activation selu (scaled exponential linear units) is an activation function that can handle small negative values with a negative log curve (neurons can still be updated and learn). It takes in inputs and gives an output that is a generalisation of the inputs.
model.add(tf.keras.layers.Activation("selu"))
# Like how we normalise our input data we also normalise the weight outputs by subtracting the mean and dividing the by the outputs standard deviation. This helps smooth out values with large weights and reduce overfitting from covariance shit.
# If we learnt on a dataset containing a bunch of white aeroplanes then if we try to generalise to unseen aeroplanes that may be blue then our model will perform bad without batch normalisation. Batch normalisation stops the colour white from being a large influence
# on how the model predicts an aeroplane.
model.add(tf.keras.layers.BatchNormalization())
# Max pooling downsamples the the fature map output from the convolutional layer, this is so we can make our feature map overfit less and be more robust to rotation, zoom etc. This is because we take the feature map and convert it to a "lower resolution" pixel map
# where features are weighted in smaller batches to allow for movement when upscaling the array. For example if we weight a line in the image as being 2 pixels wide then max pooling may downscale this to 1 pixel wide allowing for movement and less emphasis on the need
# for the picture to include the line. [2, 2] means we halve the image in both x and y direction (downscale by 2)
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
# Dropout randomly takes some of the output weights (trained neuron) and removes them from the training data (in this case we remove 30% at this step). This is because weights will eventually settle on simalar patterns they have been representing from images, so to
# combat this we randomly remove some and rely on the surrounding neurons to convey the change in the local area. This helps reduce overfitting by stopping the reliance on a single feature and make generalisation more robust.
model.add(tf.keras.layers.Dropout(0.4))

In [30]:
# Second convolutional layer.
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same"))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.4))

In [31]:
# Third convolutional layer.
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same"))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.4))

In [32]:
# Flatten, removes all the dimensions of our layers into a 1D array
model.add(tf.keras.layers.Flatten())

In [33]:
# First fully-connected layer.
# Dense takes the 1D flattened array represents all the input neurons as 512 output neurons.
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())

In [34]:
# Second fully-connected layer.
# This dense takes in the above layer and represents all the input neurons into 10 neurons which correspond to the final output of what image is classified as what.
model.add(tf.keras.layers.Dense(num_classes))
# Softmax outputs a value between 0 and 1 for how much each neuron represents of the whole, in this case this will represent the corresponding probability of what class(s) the given pixels/picture represents
model.add(tf.keras.layers.Activation("softmax"))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 64)        0         
__________

In [36]:
# ADAM optimiser is similar to stochastic gradient descent but keeps both the average training step and the covariance of the training step to avoid local minima and obtain the next best weight.
# categorical_crossentropy and accuracy are both used to return the training set loos and accuracy as well as the validation set loss and accuracy. Accuracy is a percentage of the correctly classified pictures and loss is the log sum of all the images incorrectly classified
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [37]:
# Datagen is a form of preprocessing used to change the rotation/width/height of an image so that the model can learn to be more robust and generalise for images not taken perfectly square on.
datagen = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.08, height_shift_range = 0.08, horizontal_flip = True)
datagen.fit(x_train)

In [38]:
# Batch size is how many groups the model is split up into and are run through till the model is updated.
batch_size = 64
# Epochs is the number of "full" runs from start to finish of training.
epochs = 150

In [8]:
# Reduce learning rate when the weights stop improving so we dont learn useless data
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.2, patience = 5, min_lr = 0.001)
# Steps per epoch is the amound of steps the model has to train before the epoch has completed (being a "full" run through of the dataset)
training = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), steps_per_epoch = x_train.shape[0] / batch_size, epochs = epochs, validation_data=(x_test, y_test), callbacks = [reduce_lr])
# Score tells us the final validation loss and accuracy on an unseen test set that our model can predict for.
final_score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = 1)

print("Validation loss: ", final_score[0])
print("Validation accuracy: ", final_score[1])

NameError: name 'model' is not defined

In [9]:
# Plot showing the change in training loss and validation loss over time as the amount of epochs the model goes though increases.
plt.plot(training.history["loss"])
plt.plot(training.history["val_loss"])
plt.title("Training loss and validation loss over time as the number of epochs increase")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["Training loss", "Validation loss"])
plt.show()

# Plot showing the change in accuracy of the training and accuracy of testing predictions over time as the amount of epochs the model goes though increases.
plt.plot(model.history["acc"])
plt.plot(model.history["val_acc"])
plt.title("Training accuracy and validation accuracy over time as the number of epochs increase")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Training accuracy", "Validation accuracy"])
plt.show()

NameError: name 'training' is not defined